In [ ]:
# UMAP script to reduce zircon geochemistry
#2024-10-03 MAAZ
#2025-06-26, EBL

# input CG2024 dataset and select parameters (if not already input)
# run UMAP
# create decision boundary - linear SVC
# ROC_AUC curve calculation
# Plot global geochemistry in umap



# Importing libraries and functions

In [ ]:
import os
import numpy as np
import pandas as pd
import umap
import umap.plot
# import scipy

import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc
import plotly.express as px #for plottly
import joblib #for saving UMAP model

from matplotlib import colors

#Machine learning libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc, balanced_accuracy_score, f1_score

#Advanced plotting libraries
from itertools import compress
from mpl_toolkits.mplot3d import Axes3D
from bokeh.plotting import output_notebook #for interactive plot
from matplotlib.colors import Normalize

#Script requirements
umap.plot.output_notebook() #resources= INLINE
%matplotlib inline
#%matplotlib widget #for 3d plot
sns.set_theme(style='white', context='notebook', rc={'figure.figsize':(14, 10)}) #anything smaller does not help with points

#Helper functions
def export_legend(legend, filepath2, expand=[-5,-5,5,5]):   

    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent()
    bbox = bbox.from_extents(*(bbox.extents + np.array(expand)))
    bbox = bbox.transformed(fig.dpi_scale_trans.inverted())
    
    fig.savefig(filepath2, dpi="figure", bbox_inches=bbox)

def make_dir(destDir):
    image_dir = destDir
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)  

# Loading and filtering data

In [ ]:
#User input - change directories 
data_folder1 = # input directory here

file1 = "GeoROC_apatite_v1.csv"
file2 = "legend.png" #plot
file3 = "workable_table.xlsx" #for reproducibility
file4 = "standard_scaler.xml"
file5 = "umap_model.xml" 

trial_name = 'v4_Apatite_georoc_1' #IMPORTANT: change this name to avoid overwriting outputs


#Script begins

os.chdir(data_folder1)
print(data_folder1)

data_folder2 = os.path.join(os.path.dirname(data_folder1), 'outputs', trial_name)
make_dir(data_folder2)


filepath1 = os.path.join(data_folder1, file1)
filepath2 = os.path.join(data_folder2, file2)
filepath3 = os.path.join(data_folder2, file3)
filepath4 = os.path.join(data_folder2, file4)
filepath5 = os.path.join(data_folder2, file5)

#Load table (the column indexes below can be obtained from a data dictionary)
table1 = pd.read_csv(filepath1, low_memory=False)
table1.head()

#range_imputed = list([2,3,4,5,14,19,26,29,39,54,55,56,57,58,59,60,61,62,63,64,65,66,50,51])
range_imputed = list([2,3,4,5,14,19,29,30,54,34,37,47])


#range_imputed = list([0, 1, 2,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40])
#range_imputed.extend(list([19,23,27,36,38,78,80,67,68,69,72,74,75,76,77]))
#range_imputed.extend(range(65, 77)) #imputed REE


#r1
#range_imputed.extend(list([85,90,19,97,39]))

#Generating workable table
table2 = table1.iloc[:, range_imputed] 

#medicine 1
idx1 = table2["TECTONIC SETTING"].isnull()
idx2 = table2["LOCATION"].isnull()
#idx3 = table2["LAND/SEA (SAMPLING)"].isnull()
idx4 = table2["ROCK NAME"].isnull()
#idx5 = table2["PRIMARY/SECONDARY"].isnull()
idx6 = table2["CITATION"].isnull()


table2.loc[idx1, "TECTONIC SETTING"] = 'Unknown'
table2.loc[idx2, "LOCATION"] = 'Unknown'
#table2.loc[idx3, "LAND/SEA (SAMPLING)"] = 'Unknown'
table2.loc[idx4, "ROCK NAME"] = 'Unknown'
#table2.loc[idx5, "PRIMARY/SECONDARY"] = 'Unknown'
table2.loc[idx6, "CITATION"] = 'Unknown'

#Dropping rows with empty cells

#medicine 2: do not drop string NAs
any_idx = table2.isna().any(axis=1)
table3 = table2.loc[np.invert(any_idx), :] # np.invert(any_idx)
table3.reset_index(inplace = True) #the index from the input table is preserved (for searching points)

data_start_idx = 7 #10
col_names = list(table3.columns)
a = table2.shape[0]
b = table3.shape[0]
c = col_names[data_start_idx:]

print(f"Table 2 has {a} rows and Table 3 without NA has {b} rows")
print(f"UMAP will use: {c}")
#table3.head()
table3.tail()

## Printing populations for each category

In [ ]:
categoricals = ["SAMPLE NAME",
                "TECTONIC SETTING",
                "LOCATION",
               "ROCK NAME"]
for i in range(len(categoricals)):
    temp = table3.loc[:, [categoricals[i]]].value_counts()
    print(temp)

#table3.loc[:, [categoricals[1]]].value_counts()

# Embedding space
### Run only once each time the notebook is opened. The stochastic process within UMAP wont repeat itself for all plots otherwise

Generating and saving model

In [ ]:
# run UMAP IF not using UMAP model and saved scaler from file, if these have been created already, run the cell below instead

try:
    del embedding
except:
    print('Processing for the first time')

components_output = 2 #default=2, dimensionality
neighbors_input = 20 #default=15, preservation of local (> singletons) vs global structure
min_dist_input = 0.05 #0.003, min. dist. of packing value (in low dimensional representation)

#data
data = table3.iloc[:, data_start_idx:].values

sc = StandardScaler()
scaled_data = sc.fit_transform(data)

#umap object () for umap.plot
embedding = umap.UMAP(n_neighbors= neighbors_input,
                      min_dist= min_dist_input,
                      metric='correlation', 
                      n_components= components_output,
                     ).fit(scaled_data)  

#Saving data for reproducibility
table3.to_excel(filepath3, index=False) #processed table
joblib.dump(sc, filepath4) #scaler
joblib.dump(embedding, filepath5) #umap transform

### Loading model and workable table from previous session

In [ ]:
# run this IF using the saved scaler and UMAP transform from file, OR after running cell above

sc = joblib.load(filepath4) #scaler
embedding = joblib.load(filepath5) #umap transformation

embedding2 = embedding.embedding_

#Load table 
table3 = pd.read_excel(filepath3)
table3.tail()

In [ ]:
#UMAP diagnostic plotting
fig, ax = plt.subplots(figsize=(16,8))
ax=umap.plot.diagnostic(embedding, diagnostic_type='local_dim',point_size=20, ax=ax)
plt.show()

## PCA after transformation to understand global structure preservation in UMAP

In [ ]:
# Apply centered log transformation to the dataset and create a calculated theoretical CaPo content
emb = pd.DataFrame(embedding2)


#theoretical calc (1000000ppm- sum TE)

#copy paste from list under cell 2
Sum_TE = ['SR(PPM)', 'Y(PPM)', 'Eu/Eu*', 'LA(PPM)', 'ND(PPM)', 'LU(PPM)']
Dat_UMAP = pd.concat([table3, emb.iloc[:, [0, 1]]], axis=1)
Dat_UMAP['CaPO'] = 1_000_000 - table3[Sum_TE].sum(axis=1)

#group variables and calc parameter for CLR transformation

# CLR transformation
def centered_log_ratio_transform(X):
    """Centered log-ratio transformation for compositional data."""
    if np.any(X <= 0):
        raise ValueError("CLR transformation requires strictly positive values.")
    geometric_mean = np.exp(np.mean(np.log(X), axis=1)).to_numpy().reshape(-1, 1)
    return np.log(X / geometric_mean)

# Correlation circle plot
def plot_correlation_circle(loadings, title="Correlation Circle"):
    plt.figure(figsize=(7, 7))
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
    circle = plt.Circle((0, 0), 1, color='gray', fill=False, linestyle='--')
    plt.gca().add_artist(circle)

    x_axis, y_axis = loadings.columns[0], loadings.columns[1]
    for i in range(loadings.shape[0]):
        plt.arrow(0, 0, loadings.iloc[i, 0], loadings.iloc[i, 1],
                  color='r', alpha=0.7, head_width=0.02)
        plt.text(loadings.iloc[i, 0]*1.1, loadings.iloc[i, 1]*1.1,
                 loadings.index[i], ha='center', va='center', fontsize=9)

    plt.title(title)
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.xlim(-1.1, 1.1)
    plt.ylim(-1.1, 1.1)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# PCA pipeline
def run_pca_pipeline(df, clr_columns, group_column='TECTONIC SETTING', n_components=3):
    features = df[clr_columns]
    groups = df[group_column]

    clr_scaled = pd.DataFrame(centered_log_ratio_transform(features),
                            columns=features.columns, index=features.index)

    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(clr_scaled)

    pca_scores = pd.DataFrame(principal_components,
                              columns=[f'PC{i+1}' for i in range(n_components)],
                              index=df.index)
    pca_scores[group_column] = groups

    # Scree plot
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, n_components + 1),
             pca.explained_variance_ratio_, marker='o', linestyle='--')
    plt.title("Scree Plot")
    plt.xlabel("Principal Component")
    plt.ylabel("Explained Variance Ratio")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 2D Scatter Plot: PC1 vs PC2
    plt.figure(figsize=(7, 6))
    sns.scatterplot(data=pca_scores, x='PC1', y='PC2', hue=group_column, palette='Set2')
    plt.title('PC1 vs PC2 Scatter Plot')
    plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
    plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 3D Scatter Plot
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    for group in pca_scores[group_column].unique():
        group_data = pca_scores[pca_scores[group_column] == group]
        ax.scatter(group_data['PC1'], group_data['PC2'], group_data['PC3'], label=group)
    ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
    ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
    ax.set_zlabel(f"PC3 ({pca.explained_variance_ratio_[2]*100:.1f}%)")
    ax.set_title("PC1 vs PC2 vs PC3 (3D Scatter Plot)")
    ax.legend()
    plt.tight_layout()
    plt.show()

    # Loadings
    loadings = pd.DataFrame(pca.components_.T[:, :3],
                            columns=['PC1', 'PC2', 'PC3'],
                            index=features.columns)

    plot_correlation_circle(loadings[['PC1', 'PC2']], title="Correlation Circle: PC1 vs PC2")
    plot_correlation_circle(loadings[['PC1', 'PC3']], title="Correlation Circle: PC1 vs PC3")

    print("\n📌 PCA Loadings (First 3 PCs):")
    print(loadings.round(3))

    return pca, pca_scores, loadings


if __name__ == "__main__":
    group_labels = table3['TECTONIC SETTING']
    clr_columns = ['SR(PPM)', 'Y(PPM)', 'Eu/Eu*', 'LA(PPM)', 'ND(PPM)', 'LU(PPM)',"CaPO"]
    Dat_UMAP['TECTONIC SETTING'] = pd.Series(group_labels, index=Dat_UMAP.index[:len(group_labels)])  # ensure group_labels matches df index
    pca_model, pca_scores, pca_loadings = run_pca_pipeline(Dat_UMAP, clr_columns=clr_columns, group_column='TECTONIC SETTING')

In [ ]:
#PCA to RGB to UMAP x, y


def pca_to_rgb(pca_scores):
    pcs = pca_scores[['PC1', 'PC2', 'PC3']].copy()
    pcs_norm = np.zeros_like(pcs.values)

    # PC 1-3 normalized to be from 0-1
    for i in range(pcs.shape[1]):
        norm = Normalize(vmin=pcs.iloc[:, i].min(), vmax=pcs.iloc[:, i].max())
        pcs_norm[:, i] = norm(pcs.iloc[:, i])

    # Convert to list of RGB tuples
    rgb_colors = [tuple(color) for color in pcs_norm]
    return np.array(rgb_colors)

#give colour for mapping
rgb_colors = pca_to_rgb(pca_scores)

#PCA biplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# PC1 vs PC2
axes[0].scatter(pca_scores['PC1'], pca_scores['PC2'],c=rgb_colors, s=30, alpha=0.9, edgecolor='k', linewidth=0.3)
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].set_title('PC1 vs PC2')

# PC1 vs PC3
axes[1].scatter(pca_scores['PC1'], pca_scores['PC3'],c=rgb_colors, s=30, alpha=0.9, edgecolor='k', linewidth=0.3)
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC3')
axes[1].set_title('PC1 vs PC3')

plt.tight_layout()
plt.show()


# UMAP plot using PCA colouring

col_x = Dat_UMAP.columns[-3]
col_y = Dat_UMAP.columns[-2]

plt.figure(figsize=(8, 6))
plt.scatter(Dat_UMAP[col_x], Dat_UMAP[col_y],c=rgb_colors, s=30, alpha=0.9, edgecolor='k', linewidth=0.3)
plt.xlabel(col_x)
plt.ylabel(col_y)
plt.title('UMAP Projection Colored by PCA RGB')
plt.tight_layout()
plt.show()

## Plotting UMAP plot coloured by variable, e.g. Lithology

In [ ]:
# enter variable below

variable_legend = "LITHOLOGY CODE" #e.g., Temporality, ml_classes

list_unique = table3[variable_legend].unique()
n_classes = len(list_unique)

#colourmap
mapping = {item:i for i, item in enumerate(list_unique)}
classif= table3[variable_legend].apply(lambda x: mapping[x]) #categorical array (same size as table3)
colourmap = sns.color_palette(palette= cc.glasbey_category10, n_colors = n_classes)
colours = [sns.color_palette(palette= colourmap)[x] for x in classif] #RGB triplets

colourmap_updated = colourmap #pre-allocating
for x in range(0, n_classes):
    
    idx = (classif == x)
    name = list_unique[x]

    colours_sub = colourmap[x]
    
    if name == 'Ore syn-mineral magmatism':
        colours_sub = colors.to_rgb('red')

    if name == 'Syn Mineral':
        colours_sub = colors.to_rgb('red')

    if name == 'Ore related magmatism':
        target_colour = (255, 208, 0)
        colours_sub = tuple(ti/255 for ti in target_colour)

    if name == 'Unknown':
        colours_sub = colors.to_rgb('lightgrey')

    if name == 'S Type Granite':
        colours_sub = colors.to_rgb('violet')

    colourmap_updated[x] = colours_sub

In [ ]:
# plot global projection of zircon geochemistry from CG2024

variable_legend1="LITHOLOGY CODE"

#Saving names
filepath3_new = filepath2.replace("legend.png", variable_legend1 + "_legend.png")
filepath4_new = filepath2.replace("legend.png", variable_legend1 + "_plot.png")
print(filepath4_new)

#Plot
markerSize = 6
fontSize = 18

fig = plt.figure(dpi= 200, figsize=(8,6)) #1200 , figsize=(10, 10)

for x in range(0, n_classes):

    
    idx = (classif == x)
    name = list_unique[x]
   
    # colours_sub = list(compress(colours, idx)) #required to index list
    colours_sub = np.asarray(colourmap_updated[x]).reshape(1,-1)        

    scatter = plt.scatter(embedding2[idx, 0], embedding2[idx, 1],
                              c=colours_sub, label = name,
                              s=15, alpha= .7, edgecolors= 'none')

#Settings
plt.grid(True)
plt.tight_layout()




lgnd = plt.legend(ncol=1, fontsize= fontSize, loc='center right', bbox_to_anchor=(1.4, 0.5),
                  markerscale= 5, scatterpoints=1)
export_legend(lgnd, filepath2= filepath3_new)
#lgnd.remove()
plt.show()

fig.savefig(filepath4_new, dpi="figure")
            